In [ ]:
import torch 
import time 

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch.nn as nn
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard import SummaryWriter

## 项目folder
from dataset import SeqMultiLabelDataset
from process import Schema2Label 
from evaluation import extract_multilabel, multilabel_inference, multilabel_evaluation

from src.dataset.converter import data_loader 
from src.train_utils import set_seed, ModelSave, get_torch_device, EarlyStop, TrainParams
from src.models.bert import BertClassifier
from src.metrics import multilabel_metrics, multilabel_log
import transformers 
from transformers import BertTokenizer
transformers.logging.set_verbosity_error()
device = get_torch_device()

In [ ]:
DIR = './trainsample'
schema = Schema2Label(DIR+'/duee_event_schema.json')
tp = TrainParams(
    log_steps = 50,
    save_steps = 10,
    epoch_size=20,
    max_seq_len=400,
    batch_size=20,
    loss_fn=nn.BCEWithLogitsLoss(),
    lr=5e-5,
    weight_decay=0.0,
    epsilon=1e-6,
    warmup_steps=100,
    dropout_rate=0.2,
    gradient_clip=5.0,
    early_stop_params = {
        'monitor':'acc',
        'mode':'max',
        'min_delta': 0,
        'patience':3,
        'verbose':False
    },
    pretrain_model = 'bert-base-chinese',
    continue_train=False,
    label2idx = schema.event_label,
    idx2label = {j:i for i,j in schema.event_label.items()},
    label_size=len(schema.event_label),
    num_layers=1
)

In [ ]:
tokenizer = BertTokenizer.from_pretrained(tp.pretrain_model, do_lower_case=True)
train_dataset = SeqMultiLabelDataset(data_loader(DIR+'/train_event.txt'),
                                tokenizer, tp.max_seq_len, tp.label2idx)
valid_dataset = SeqMultiLabelDataset(data_loader(DIR+'/valid_event.txt'), 
                                tokenizer, tp.max_seq_len,  tp.label2idx)
train_sampler = RandomSampler(train_dataset)
valid_sampler = SequentialSampler(valid_dataset)
train_loader = DataLoader(train_dataset, sampler=train_sampler, batch_size=tp.batch_size)
valid_loader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=tp.batch_size)

In [ ]:
tp.update({'num_train_steps': int(len(train_loader)*tp.epoch_size)})

CKPT = './checkpoint/event'
saver = ModelSave(CKPT, continue_train=False)
saver.init()
es = EarlyStop(**tp.early_stop_params)
global_step = 0
tb = SummaryWriter(CKPT)

model = BertClassifier(tp)
model.to(device)
optimizer, scheduler = model.get_optimizer()

In [ ]:
for epoch_i in range(tp['epoch_size']):
    print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10}  | {'Elapsed':^9}")
    print("-"*60)

    # Measure the elapsed time of each epoch
    t0_epoch, t0_batch = time.time(), time.time()
    total_loss, batch_loss, batch_counts = 0, 0, 0

    model.train()
    for step, batch in enumerate(train_loader):
        global_step +=1
        batch_counts +=1

        #Forward propogate
        model.zero_grad()
        feature = {k:v.to(device) for k, v in batch.items()}
        logits = model(feature)
        loss = model.compute_loss(feature, logits)
        batch_loss += loss.item()
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), tp.gradient_clip)
        optimizer.step()
        scheduler.step()
        # Log steps for train loss logging
        if (step % tp.log_steps == 0 and step != 0) or (step == len(train_loader) - 1):
            time_elapsed = time.time() - t0_batch
            print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^9} | {time_elapsed:^9.2f}")
            tb.add_scalar('loss/batch_train', batch_loss / batch_counts, global_step=global_step)
            batch_loss, batch_counts = 0, 0
            t0_batch = time.time()

    # On Epoch End: calcualte train & valid loss and log overall metrics
    time_elapsed = time.time() - t0_epoch
    val_metrics = multilabel_metrics(model, valid_loader, device)
    avg_train_loss = total_loss / step

    print("-"*70)
    print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_metrics['val_loss']:^10.6f} | {time_elapsed:^9.2f}")
    multilabel_log(epoch_i, val_metrics)
    print("\n")
    if es.check(val_metrics):
        break 

## Test Inference

In [ ]:
test_dataset = SeqMultiLabelDataset(data_loader(DIR+'/test_event.txt'), 
                                tokenizer, tp.max_seq_len,  tp.label2idx)
test_sampler = SequentialSampler(test_dataset)
test_loader = DataLoader(test_dataset, sampler=test_sampler, batch_size=tp.batch_size)

In [ ]:
test = pd.read_csv(DIR+'/test.csv')
valid = pd.read_csv(DIR+"/valid.csv")
pred = multilabel_inference(model, test_loader, device)
test['prob'] = pred
pred = multilabel_inference(model, valid_loader, device)
valid['pred'] = pred
valid['prob'] = valid['pred']
valid['pred_label'] = valid['pred'].map(lambda x: extract_multilabel(x, tp.idx2label, 0.5))
test['prob'] = test['pred']
test['pred_label'] = test['pred'].map(lambda x: extract_multilabel(x, tp.idx2label, 0.5))

In [ ]:
valid.to_csv('./trainsample/valid_event_cls_pred.csv')
test.to_csv('./trainsample/test_event_cls_pred.csv')

## Evaluation

In [ ]:
valid = pd.read_csv('./trainsample/valid_event_cls_pred.csv')
test = pd.read_csv('./trainsample/test_event_cls_pred.csv')

In [ ]:
valid['pred_label'] = valid['pred_label'].map(lambda x: ast.literal_eval(x))
valid['event_label'] = valid['event_label'].map(lambda x: ast.literal_eval(x))

In [ ]:
event_evaluation(valid['event_label'].values, valid['pred_label'].values)